<a href="https://colab.research.google.com/github/chaitsugale/Style-Transfer-using-CycleGAN/blob/main/Style_Transfer_MR1_T1_T2_Cyclic_GAN_Capstone_Project_Chaitanya.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Style Transfer Using Cyclic GAN Capstone Project

### By: Chaitanya Ugale

**Background** :

Misdiagnosis in the medical field is a very serious issue but it’s also uncomfortably common to occur. Imaging procedures in the medical field requires an expert radiologist’s opinion since interpreting them is not a simple binary process ( Normal or Abnormal). Even so, one radiologist may see something that another does not. This can lead to conflicting reports and make it difficult to effectively recommend treatment options to the patient.

One of the complicated tasks in medical imaging is to diagnose MRI(Magnetic Resonance Imaging). Sometimes to interpret the scan, the radiologist needs different variations of the imaging which can drastically enhance the accuracy of diagnosis by providing practitioners with a more comprehensive understanding.

But to have access to different imaging is difficult and expensive. With the help of deep learning, we can use style transfer to generate artificial MRI images of different contrast levels from existing MRI scans. This will help to provide a better diagnosis with the help of an additional image.

**Problem Statement :**

To build a Generative adversarial model(modified U-Net) which can generate artificial MRI images of different contrast levels from existing MRI scans.

### Importing Packages

In [3]:
# Importing all the required libraries

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from skimage.transform import resize
import imageio
import glob
import cv2
import pathlib

# importinf tensorflow libraries

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers,Sequential,optimizers,metrics,Model
from tensorflow.keras.utils import image_dataset_from_directory
from tensorflow.keras.utils import plot_model
from tensorflow.keras.layers import Dense,Dropout,BatchNormalization,Activation,Flatten,Conv2D,Conv2DTranspose
from keras.utils.vis_utils import plot_model

In [4]:
#loading the package for google drive

from google.colab import drive

#mounting content from google drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [7]:
#unzipping the file

!unrar x "gdrive/MyDrive/MRI+T1_T2+Dataset.RAR" "/content/gdrive/MyDrive/"

#!unzip gdrive/MyDrive/MRI+T1_T2+Dataset -d '/content/gdrive/MyDrive/'


UNRAR 5.50 freeware      Copyright (c) 1993-2017 Alexander Roshal


Extracting from gdrive/MyDrive/MRI+T1_T2+Dataset.RAR

Creating    /content/gdrive/MyDrive/Tr1                               OK
Extracting  /content/gdrive/MyDrive/Tr1/.DS_Store                          0%  OK 
Creating    /content/gdrive/MyDrive/Tr1/TrainT1                       OK
Extracting  /content/gdrive/MyDrive/Tr1/TrainT1/Image #10.png              1%  OK 
Extracting  /content/gdrive/MyDrive/Tr1/TrainT1/Image #11.png              2%  OK 
Extracting  /content/gdrive/MyDrive/Tr1/TrainT1/Image #12.png              3%  OK 
Extracting  /content/gdrive/MyDrive/Tr1/TrainT1/Image #13.png              4%  OK 
Extracting  /content/gdrive/MyDrive/Tr1/TrainT1/Image #14.png              5%  OK 
Extracting  /content/gdrive/MyDrive/Tr1/TrainT1/Image #15.png              6%  OK 
Extracting  /content/gdrive/MyDrive/Tr1/TrainT1/Image #16.png              7% 

Counts of File